In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
#train = pd.read_csv("C:\\Users\\megpamid\\Downloads\\250k_rndm_zinc_drugs_clean_3.csv\\drug_dataset.csv")

train = pd.read_csv("C:\\Users\\nagkiran3\\Downloads\\Working\\Drug Discovery using GAI\\data\\drug_dataset.csv")

In [4]:
train                                                                       

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182
...,...,...,...,...
249450,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc...,3.36790,0.745901,2.900726
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,2.87430,0.799426,2.326627
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1\n,2.90054,0.878086,1.840642
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1\n,2.55624,0.852917,2.024638


In [5]:
train.dropna(inplace=True)          

In [6]:
num_input_features = train.shape[1]  # The number of columns in the DataFrame

In [7]:
print("Number of input features:", num_input_features)

Number of input features: 4


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)

In [10]:
!pip install tensorflow
#import tensorflow as tf

Could not fetch URL https://pypi.org/simple/tensorflow/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/tensorflow/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006)'))) - skipping


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [11]:
pip install --upgrade pip --trusted-host pypi.org --trusted-host files.pythonhosted.org


Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


In [35]:
latent_dim = 64

In [36]:
import keras as keras

In [37]:
from keras.models import Sequential

In [38]:
from keras.layers import Dense, Dropout, Flatten

In [39]:
input_shape = 4

In [40]:
# Define the encoder
encoder_inputs = tf.keras.layers.Input(shape=(input_shape,))

In [41]:
# Define encoder layers
encoder_outputs = tf.keras.layers.Dense(64, activation='relu')(encoder_inputs)

In [42]:
z_mean = tf.keras.layers.Dense(latent_dim, name='z_mean')(encoder_outputs)

In [43]:
z_log_var = tf.keras.layers.Dense(latent_dim, name='z_log_var')(encoder_outputs)

In [44]:
# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = tf.keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])

In [45]:
# Define the decoder
decoder_inputs = tf.keras.layers.Input(shape=(latent_dim,))
# Define decoder layers
decoder_outputs = tf.keras.layers.Dense(input_shape, activation='sigmoid')(decoder_inputs)

In [49]:
decoder = tf.keras.models.Model(inputs=decoder_inputs, outputs=decoder_outputs)

In [50]:
encoder_decoder_outputs = decoder(z)

In [51]:
# Define VAE model
vae = tf.keras.models.Model(encoder_inputs, encoder_decoder_outputs)

In [52]:
# Compile VAE model
vae.compile(optimizer='adam', loss='mse')  # Adjust loss function as needed

In [53]:
# Print model summary
vae.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 64)           320         ['input_3[0][0]']                
                                                                                                  
 z_mean (Dense)                 (None, 64)           4160        ['dense_2[0][0]']                
                                                                                                  
 z_log_var (Dense)              (None, 64)           4160        ['dense_2[0][0]']                
                                                                                            